In [1]:
import json
import pandas as pd

from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [2]:
#location = "boston"
#location = "geneva"
location = "hong-kong"
index_name = "airbnb_history_" + location

In [3]:
from elasticsearch.helpers import scan

res = scan(
    es,
    index=index_name,
    query={"query": { "match_all" : {}}}
)

In [4]:
data = list(res)

In [5]:
len(data)

303811

In [6]:
# appending rows 
docs = []
for item in data:  
    doc = {
        'id': item['_id'],
        'index': item['_index'],
        'listing_url': item['_source']['listing_url'],
        'scrape_id': item['_source']['scrape_id'],
        'last_scraped': item['_source']['last_scraped'],
        'name': item['_source']['name'],
        'host_id': item['_source']['host_id'],
        'price': item['_source']['price'],
        'availability_30': item['_source']['availability_30'],
        'availability_60': item['_source']['availability_60'],
        'availability_90': item['_source']['availability_90'],
        'availability_365': item['_source']['availability_365'],
        'first_review': item['_source']['first_review'],
        'last_review': item['_source']['last_review'],
        'review_scores_rating': item['_source']['review_scores_rating'],
        'review_scores_accuracy': item['_source']['review_scores_accuracy'],
        'review_scores_cleanliness': item['_source']['review_scores_cleanliness'],
        'review_scores_checkin': item['_source']['review_scores_checkin'],
        'review_scores_communication': item['_source']['review_scores_communication'],
        'review_scores_location': item['_source']['review_scores_location'],
        'review_scores_value': item['_source']['review_scores_value'],
        'overall_rating': item['_source']['overall_rating']
    }
    
    docs.append(doc) 

In [11]:
len(docs)

303811

In [17]:
def divide_chunks(docs, n): 
    """ Split list of documents into n-sized chunks
    :param l: list
    :param n: num of paritions
    """
    org_size = len(docs)
    chunk_size = org_size / float(n)
    print(org_size)
    print(chunk_size)
    
    return [ docs[int(round(chunk_size * i)): int(round(chunk_size * (i + 1)))] for i in range(n) ]

In [18]:
partitions = divide_chunks(docs, 3)

303811
101270.33333333333


In [19]:
len(partitions[0])

101270

In [22]:
partitions[0][:2]

[{'id': 'FuxpP3cBYxiStLCpn4Vo',
  'index': 'airbnb_history_hong-kong',
  'listing_url': 'https://www.airbnb.com/rooms/69074',
  'scrape_id': 20180413183246,
  'last_scraped': '20180413',
  'name': 'Beautiful oasis of plants & art and best location',
  'host_id': 160139,
  'price': 1413.0,
  'availability_30': 9,
  'availability_60': 23,
  'availability_90': 53,
  'availability_365': 144,
  'first_review': '20110214',
  'last_review': '20180408',
  'review_scores_rating': 98.0,
  'review_scores_accuracy': 10.0,
  'review_scores_cleanliness': 10.0,
  'review_scores_checkin': 10.0,
  'review_scores_communication': 10.0,
  'review_scores_location': 10.0,
  'review_scores_value': 9.0,
  'overall_rating': 4.916666666666667},
 {'id': 'F-xpP3cBYxiStLCpn4Vo',
  'index': 'airbnb_history_hong-kong',
  'listing_url': 'https://www.airbnb.com/rooms/101765',
  'scrape_id': 20180413183246,
  'last_scraped': '20180413',
  'name': 'Lamma Island flat 2min walk from the beach!',
  'host_id': 532909,
  'pr

In [20]:
len(partitions[1])

101271

In [21]:
len(partitions[2])

101270

In [23]:
import json
i = 0

for p in partitions:
    
    with open('airbnb_' + location + "_" + str(i) + '.json', 'w') as f:
        json.dump(p, f)
        
    i = i+1